<a href="https://colab.research.google.com/github/nyp-sit/iti107-2024S2/blob/main/session-4/word_embedding_glove.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Using Pre-trained Contex-free Word Embedding

In this lab exercise, we will use a pretrained word embedding (GloVE) for our text classification task, instead of training our own embedding layer.

## Setup

In [ ]:
import os
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split

### Download the IMDb Dataset

Download the dataset using Keras file utility and do the clean-up as before

In [ ]:
# downloaded the datasets.
train_data_url = 'https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/datasets/imdb_train.csv'
test_data_url = 'https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/datasets/imdb_test.csv'

train_df = pd.read_csv(train_data_url)
test_df = pd.read_csv(test_data_url)

We will just use a subset of 10000 for training/validation and 500 for testing.

In [ ]:
TRAIN_SIZE = 10000
TEST_SIZE = 500
BATCH_SIZE = 1000

train_df = train_df.sample(n=TRAIN_SIZE, random_state=128)
test_df = test_df.sample(n=TEST_SIZE, random_state=128)

# convert the text label to numeric label
train_df['sentiment'] =  train_df['sentiment'].apply(lambda x: 0 if x == 'negative' else 1)
test_df['sentiment'] =  test_df['sentiment'].apply(lambda x: 0 if x == 'negative' else 1)

In [ ]:
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=128)

Let's convert to pandas dataframe to Tensorflow Dataset (tf.data.Dataset) suitable for model training later.

In [ ]:
train_ds = tf.data.Dataset.from_tensor_slices(
            (train_df['review'].values,
            train_df['sentiment'].values)
)

val_ds = tf.data.Dataset.from_tensor_slices(
            (val_df['review'].values,
            val_df['sentiment'].values)
)

test_ds = tf.data.Dataset.from_tensor_slices(
            (test_df['review'].values,
            test_df['sentiment'].values)
)

# optimize the data pipeline
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.batch(BATCH_SIZE).cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.batch(BATCH_SIZE).cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.batch(BATCH_SIZE).cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
len(train_df)

## Text preprocessing

We then initialize a TextVectorization layer with the desired parameters to vectorize our movie reviews.

In [ ]:
# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
MAX_SEQUENCE_LENGTH = 500

# Use the text vectorization layer to normalize, split, and map strings to
# integers.
# Set output_sequence length as all samples are not of the same length.
vectorize_layer = tf.keras.layers.TextVectorization(
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=MAX_SEQUENCE_LENGTH)

# Make a text-only dataset (no labels) and call adapt to build the vocabulary.
text_ds = train_ds.map(lambda x, y: x)
vectorize_layer.adapt(text_ds)

## Getting the pre-trained embedding model

We will use the pre-trained GloVe embeddings available from [stanford site](https://nlp.stanford.edu/projects/glove/). The original zip file contains embedding of different dimensions (e.g. 50d, 100d, 200d, etc) and is more than 800MB in file size. To save downloading time, we have made a copy of 50d GloVe file available on our course website for download. If you want to experiment with other embedding dimensions, please feel free to download from the stanford site.


In [ ]:
glove_url = 'https://nyp-aicourse.s3-ap-southeast-1.amazonaws.com/pretrained-models/glove.6B.50d.zip'

glove_files = tf.keras.utils.get_file("glove.6B.50d.zip", glove_url,
                                    extract=True, cache_dir='.',
                                    cache_subdir='')

## Load the Embedding layer

In the code below, we read the embeddings from the download file line by line to create the embeddings index and then initialize the Keras embedding layer with this embeddings index.

In [ ]:
# Load up the GloVe word embedding data
EMBEDDING_DIM = 50
print("Loading GloVe Word Embedding...")
embeddings_index = {}

with open('glove.6B.50d.txt', encoding="utf8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
    f.close()

Let's print out the embedding for the word `happy`.

In [ ]:
embeddings_index['happy']

In [ ]:
# Construct the word embedding matrix that will be used in the Embedding layer.
vocab = vectorize_layer.get_vocabulary()
glove_embedding_matrix = np.zeros((len(vocab), EMBEDDING_DIM))
for i, word in enumerate(vocab):
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        glove_embedding_matrix[i] = embedding_vector

In [ ]:
vocab_size = len(vocab)
embedding_layer = tf.keras.layers.Embedding(VOCAB_SIZE,
                            EMBEDDING_DIM,
                            input_length = MAX_SEQUENCE_LENGTH,
                            weights=[glove_embedding_matrix],
                            trainable=False)

In [ ]:
print(vocab_size)

## Create a classification model

We will now create the model as before, but this time with embedding layer initialized with pretrained embedding.

In [ ]:
model = tf.keras.Sequential([
    vectorize_layer,
    embedding_layer,
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

## Compile and train the model

In [ ]:
import os
root_logdir = os.path.join(os.curdir, "tb_logs")

def get_run_logdir():    # use a new directory for each run
	import time
	run_id = time.strftime("run_%Y_%m_%d-%H_%M_%S")
	return os.path.join(root_logdir, run_id)

run_logdir = get_run_logdir()
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=run_logdir)
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath="bestcheckpoint.weights.h5",
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)


Compile and train the model using the `Adam` optimizer and `BinaryCrossentropy` loss.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
              metrics=['accuracy'])

In [ ]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=100,
    callbacks=[tensorboard_callback, model_checkpoint_callback])

In [ ]:
%load_ext tensorboard
%tensorboard --logdir tb_logs

With pretrained embedding layer, we reaches a validation accuracy of around 70%, worse than jointly train our embedding layer with the classification task.

This maybe because the kind of vocabulary used to train the pretrained embedding is quite different from the one used in the IMDB dataset.

If we have enough data (like in our case), joinly train our own embedding layer will usually yield a better performance.


Let's evaluate the model on our test dataset.

In [ ]:
model.load_weights("bestcheckpoint.weights.h5")
model.evaluate(test_ds)

## Additional Exercise

You can try other pretrained embedding available from the GloVe website, such as those trained on Common Crawl dataset (Beware: this pretrained embedding is huge, i.e. more than 1.7GB)